## Introduction:
This notebook is to apply two layers DNN on MNIST dataset 

Classification steps:
0. Input - MNIST dataset
1. Convolutional and Max-Pooling
2. Convolutional and Max-Pooling
3. Fully Connected Layer
4. Processing - Dropout
5. Readout layer - Fully Connected
6. Outputs - Classified digits

In [7]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [5]:
sess = tf.Session()

In [6]:
sess = tf.InteractiveSession()

## Load images:

In [8]:
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [9]:
mnist

## Placeholders:

In [10]:
x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None, 10]) # true/ real class

## Initialize weights and biases:
we gonna init with small random numbers instead of zeros !

In [11]:
## weights
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

In [12]:
## Biases
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

## Convolutional layer: 
Filtering phase

In [14]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides= [1,1,1,1], padding='SAME')

## Max pooling:

- Kernel size: 2x2 (if the image is a 2x2 matrix, it would result in one output pixel)
- Strides: dictates the sliding behaviour of the kernel. In this case it will move 2 pixels everytime, thus not overlapping.

In [15]:
def max_pooling_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

## First convolutional and max-pooling layer:

- init weights and biases 
- Size of the filter/kernel: 5x5; Input channels: 1 (greyscale); 32 feature maps

In [16]:
w_conv1 = weight_variable([5, 5, 1, 32])

In [17]:
b_conv1 = bias_variable([32])

## Converting images of the data set to tensors:

28 pixels by 28 pixels and 1 channel (grayscale)

In this case the first dimension is the ID number of the image (position of the input on the batch) and can be of any size (due to -1)


In [18]:
x_image = tf.reshape(x, [-1, 28, 28, 1])

## Convolution:

In [19]:
convolv_image = conv2d(x_image, w_conv1) + b_conv1

In [20]:
convolv_image

<tf.Tensor 'add:0' shape=(?, 28, 28, 32) dtype=float32>

## Apply RELU: 
RELU is the most common used activation function

In [21]:
h_conv1 = tf.nn.relu(convolv_image)

In [22]:
h_conv1

<tf.Tensor 'Relu:0' shape=(?, 28, 28, 32) dtype=float32>

## Apply the max pooling on the regularized image data:

In [23]:
h_pool1 = max_pooling_2x2(h_conv1)

In [24]:
h_pool1

<tf.Tensor 'MaxPool:0' shape=(?, 14, 14, 32) dtype=float32>

In [25]:
layer1 = h_pool1

## 2nd convolutional and max-pooling layer:

- We gonna repeat the above steps again .. starting from init the weights and biases !
- Filter/kernel: 5x5 (25 pixels) ; Input channels: 32 (from the 1st Conv layer, we had 32 feature maps); 64 output feature maps

In [26]:
w_conv2 = weight_variable([5, 5, 32, 64])

In [28]:
b_conv2 = bias_variable([64])

In [29]:
convolv_image_2 = conv2d(layer1, w_conv2) + b_conv2  ## convolution 

In [30]:
h_conv2 = tf.nn.relu(convolv_image_2) ## RELU

In [31]:
## max pooling
h_pool2 = max_pooling_2x2(h_conv2)

In [32]:
layer2 = h_pool2

## Third and final layer:
Fully connected layer

### Weights and Biases:
Composition of the feature map from the last layer (7x7) multiplied by the number of feature maps (64); 1024 outputs to Softmax layer

In [34]:
w_fc1 = weight_variable([7*7*64, 1024])

In [35]:
b_fc1 = bias_variable([1024])

## Flattening Second Layer:

In [36]:
layer2_matrix = tf.reshape(layer2, [-1, 7*7*64])

In [37]:
layer2_matrix

<tf.Tensor 'Reshape_1:0' shape=(?, 3136) dtype=float32>

In [39]:
## matrix multi

matmul_fc1=tf.matmul(layer2_matrix, w_fc1) + b_fc1

In [40]:
## RELU

h_fc1 = tf.nn.relu(matmul_fc1)

In [41]:
layer3 = h_fc1

## Optional phase for reducing overfitting - Dropout:

In [42]:
keep_prob = tf.placeholder(tf.float32)

In [43]:
layer3_drop = tf.nn.dropout(layer3, keep_prob)

## Init weights and biases:
- we gonna repeat the steps after applying dropout 
- Input channels: 1024 (neurons from the 3rd Layer); 10 output features

In [44]:
w_fc2 = weight_variable([1024, 10])

In [45]:
b_fc2 = bias_variable([10])

In [47]:
## building the model 
matmul_fc2=tf.matmul(layer3_drop, w_fc2) + b_fc2

In [48]:
## softmax 
y_conv = tf.nn.softmax(matmul_fc2)

In [49]:
y_conv

<tf.Tensor 'Softmax:0' shape=(?, 10) dtype=float32>

In [50]:
layer4 = y_conv

## Define the loss function and model training:

In [51]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_*tf.log(layer4), reduction_indices=[1]))

In [52]:
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

In [53]:
correct_prediction = tf.equal(tf.argmax(layer4,1), tf.argmax(y_,1))

In [54]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [55]:
accuracy

<tf.Tensor 'Mean_1:0' shape=() dtype=float32>

# Run the model:

In [56]:
sess.run(tf.initialize_all_variables())

In [62]:
for i in range(1100):
    batch = mnist.train.next_batch(50)
    train_accurcy = accuracy.eval(feed_dict={x: batch[0], y_: batch[1], keep_prob: 1.0})
    if i%100 == 0:
        print(i, float(train_accurcy))
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

(0, 0.07999999821186066)
(100, 0.8199999928474426)
(200, 0.8600000143051147)
(300, 0.8999999761581421)
(400, 0.9200000166893005)
(500, 0.9599999785423279)
(600, 0.8999999761581421)
(700, 0.9399999976158142)
(800, 0.9399999976158142)
(900, 0.8999999761581421)
(1000, 0.9800000190734863)


## Testing the model:

In [63]:
print("test accuracy %g"%accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

test accuracy 0.9682


In [64]:
sess.close() #finish the session